### Testing

In [6]:
%cd ..

/Users/cz938/Library/CloudStorage/Box-Box/NYUAD/Research/mml_flood


/Users/cz938/Library/CloudStorage/Box-Box/NYUAD/Research/mml_flood/venv_mml/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.


In [2]:
# !pip install xgboost

In [3]:
import pandas as pd
# import utils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [4]:

def baseline(y_test, results):
	y_base = np.zeros(len(y_test)) 
	print('Dumb baseline AUC', metrics.roc_auc_score(y_test,y_base))
	results['base1']=utils.get_scores_clf(y_test,y_base)

def stats_only(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, :25], np.ravel(y_train), x_test.iloc[:, :25]) #Riley: generally not a good idea to hardcode numbers like this, better we keep track by having 'nlp_#' on feature names
	results['stats only'] = utils.get_scores_clf(y_test, y_pred_prob)

def nlp_only(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, 25:153], np.ravel(y_train), x_test.iloc[:, 25:153])
	results['nlp only'] = utils.get_scores_clf(y_test, y_pred_prob)

def weather_only(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, 153:], np.ravel(y_train), x_test.iloc[:, 153:])
	results['weather only'] = utils.get_scores_clf(y_test, y_pred_prob)
 
def stats_nlp(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, :153], np.ravel(y_train), x_test.iloc[:, :153])
	results['stats nlp'] = utils.get_scores_clf(y_test, y_pred_prob)

def weather_nlp(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, 25:], np.ravel(y_train), x_test.iloc[:, 25:])
	results['weather nlp'] = utils.get_scores_clf(y_test, y_pred_prob)
 
def stats_weather(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.drop(columns=x_train.columns[25:25+128]), np.ravel(y_train), x_test.drop(columns=x_train.columns[25:25+128]))
	results['stats weather'] = utils.get_scores_clf(y_test, y_pred_prob)

def all_vars(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['all vars'] = utils.get_scores_clf(y_test, y_pred_prob)

# N = 1

In [ ]:
xy_df_sub1 = pd.read_csv('data/xy_df/xy_df_sub1_weather_full.csv')
xy_df_sub1 = xy_df_sub1.drop([ 'latitude', 'longitude', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'label', 'Unnamed: 0', ], axis=1)
x = xy_df_sub1.drop(xy_df_sub1.filter(regex='target').columns, axis=1)#drop target col
x = x.select_dtypes(['number'])#drop index col
y = xy_df_sub1.filter(regex='target') #filter to cols containing target

column_index = x.columns.get_loc('year')
x = x.iloc[:, column_index+1:]
x_train1, x_test1, y_train1, y_test1 = train_test_split(x, y, test_size=0.3, random_state=42) # Riley: something to think, I think I didn't do random shuffle but kept chronologically 

print(x_train1.shape, x_test1.shape, y_train1.shape, y_test1.shape)

(33783, 633) (14479, 633) (33783, 1) (14479, 1)


In [13]:
xy_df_sub1.head(5)

,Unnamed: 0.1,grid_id,year,flood_amt,storm_amt,earthquake_amt,extreme temperature _amt,landslide_amt,volcanic activity_amt,drought_amt,...,feature_470,feature_471,feature_472,feature_473,feature_474,feature_475,feature_476,feature_477,feature_478,feature_479
0,0,"(45, 348)",1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.065926,-1.259714,-1.017257,-0.316835,-1.078647,-0.714023,-0.494092,-1.079635,-0.884315,-0.278880
1,1,"(45, 348)",1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.827393,-1.128178,-0.851815,-0.948912,-0.720752,-0.325468,-0.686827,-1.966628,-1.372923,-1.526947
2,2,"(45, 348)",1970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.286424,-1.477790,-0.369442,-0.694219,-1.491534,-1.156600,-1.244941,-1.335228,-0.803331,-1.001159
3,3,"(45, 350)",2001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.645275,-0.927491,-0.782172,-1.245552,-0.693470,-0.964625,-1.113105,-0.475281,-0.160082,-0.303555
4,4,"(56, 123)",2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.001294,1.630767,0.898941,-0.054849,0.669643,-0.322016,0.956401,0.335617,-0.508404,-0.685915


In [11]:
results1 = {}

In [12]:
baseline(y_test1, results1)

Dumb baseline AUC 0.5


NameError: name 'utils' is not defined

In [ ]:
stats_only(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.9755363036691113
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6145012437639166 0.6
auc, f1, accu, accu_bl, precision, recall=  0.9589326370081033 0.5862548586881776 0.9520685130188549 0.7462431188904666 0.09597767885780048 0.5339366515837104
[[13667   591]
 [  103   118]]


In [ ]:
nlp_only(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.8753095143386883
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 20}
maximum f1 score, thres 0.5563224804334327 0.6
auc, f1, accu, accu_bl, precision, recall=  0.7042259041363774 0.5559558099639905 0.9603563782029145 0.5856255026153453 0.032088957145297796 0.19909502262443438
[[13861   397]
 [  177    44]]


In [35]:
weather_only(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.9990863676794355
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5377543506730906 0.4
auc, f1, accu, accu_bl, precision, recall=  0.7093306036334923 0.5002709577651027 0.9807307134470613 0.5269219344351571 0.023469274442286067 0.058823529411764705
[[14187    71]
 [  208    13]]


In [ ]:
stats_nlp(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.9888364092415316
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6394274104608029 0.6
auc, f1, accu, accu_bl, precision, recall=  0.9632748844976449 0.6353185734470012 0.9649837695973479 0.7282997113948572 0.11148433165336347 0.4841628959276018
[[13865   393]
 [  114   107]]


In [37]:
weather_nlp(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.9838890228633794
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5705445735285974 0.4
auc, f1, accu, accu_bl, precision, recall=  0.7127926593881723 0.5051003539858206 0.9735478969542095 0.572277118061528 0.036738373903134586 0.1583710407239819
[[14061   197]
 [  186    35]]


In [ ]:
stats_weather(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.9999673608397704
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.652524130843805 0.4
auc, f1, accu, accu_bl, precision, recall=  0.9621782547735366 0.6089128544323812 0.9725809793494026 0.7009739392158344 0.11695018678473228 0.42081447963800905
[[13989   269]
 [  128    93]]


In [39]:
all_vars(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.9999676543573984
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6408646720163109 0.4
auc, f1, accu, accu_bl, precision, recall=  0.9601178095459943 0.6022015538877009 0.9729953726086056 0.6766832179314748 0.10069394094353928 0.37104072398190047
[[14006   252]
 [  139    82]]


# N = 2

In [49]:
xy_df_sub2 = pd.read_csv('data/xy_df/xy_df_sub_2.csv')
xy_df_sub2 = xy_df_sub2.drop([ 'latitude', 'longitude', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'label', 'Unnamed: 0'], axis=1)
x = xy_df_sub2.drop(xy_df_sub2.filter(regex='target').columns, axis=1)#drop target col
x = x.select_dtypes(['number'])#drop index col
y = xy_df_sub2.filter(regex='target') #filter to cols containing target

column_index = x.columns.get_loc('year')
x = x.iloc[:, column_index+1:]
x_train2, x_test2, y_train2, y_test2 = train_test_split(x, y, test_size=0.3, random_state=42)

In [50]:
results2 = {}

baseline(y_test2, results2)

Dumb baseline AUC 0.5
maximum f1 score, thres 0.4965402134983831 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5 0.4965402134983831 0.9862559569031011 0.5 0.013744043096898957 0.0
[[14280     0]
 [  199     0]]


In [51]:
stats_only(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.9818527874075745
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6605495735305698 0.7
auc, f1, accu, accu_bl, precision, recall=  0.9735614346241008 0.6605495735305698 0.9772774362870364 0.6986071815660938 0.12300905196812514 0.4120603015075377
[[14068   212]
 [  117    82]]


In [ ]:
nlp_only(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.8888252532198024
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5602885757638602 0.5
auc, f1, accu, accu_bl, precision, recall=  0.731011324127641 0.5151082322584672 0.9693348988189792 0.5781357769238349 0.030868904305428733 0.17587939698492464
[[14000   280]
 [  164    35]]


In [53]:
weather_only(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.9993584532944344
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 20}
maximum f1 score, thres 0.5618658077044751 0.4
auc, f1, accu, accu_bl, precision, recall=  0.7213791295412638 0.5324516819157687 0.9794875336694523 0.5535522500457469 0.03061589294555441 0.11557788944723618
[[14159   121]
 [  176    23]]


In [ ]:
weather_nlp(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.9840353444350358
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5722210017235636 0.4
auc, f1, accu, accu_bl, precision, recall=  0.7296244879861493 0.5156429822148172 0.9768630430278334 0.5720421083006066 0.035992636149821 0.15577889447236182
[[14113   167]
 [  168    31]]


In [ ]:
stats_nlp(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.9899113290337396
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6639083141366179 0.7
auc, f1, accu, accu_bl, precision, recall=  0.9713295117041791 0.6639083141366179 0.9781752883486429 0.6965848148304548 0.12632123677232232 0.40703517587939697
[[14082   198]
 [  118    81]]


In [ ]:
stats_weather(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.9999905079270226
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6514583563554209 0.4
auc, f1, accu, accu_bl, precision, recall=  0.9693254437453374 0.6041131111846925 0.9794184681262518 0.6650069324212097 0.10792495171042518 0.3417085427135678
[[14113   167]
 [  131    68]]


In [57]:
all_vars(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.9999882246104987
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6590628045920017 0.4
auc, f1, accu, accu_bl, precision, recall=  0.9681726208071169 0.5906581345301145 0.9785896816078459 0.6844071548217276 0.11885675621723797 0.38190954773869346
[[14093   187]
 [  123    76]]


# N = 5

In [62]:
xy_df_sub5 = pd.read_csv('xy_df_sub_5_combined.csv')
xy_df_sub5 = xy_df_sub5.drop([ 'latitude', 'longitude', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'label', 'Unnamed: 0'], axis=1)
x = xy_df_sub5.drop(xy_df_sub5.filter(regex='target').columns, axis=1)#drop target col
x = x.select_dtypes(['number'])#drop index col
y = xy_df_sub5.filter(regex='target') #filter to cols containing target

column_index = x.columns.get_loc('year')
x = x.iloc[:, column_index+1:]
x_train5, x_test5, y_train5, y_test5 = train_test_split(x, y, test_size=0.3, random_state=42)

print(x_train5.shape, x_test5.shape, y_train5.shape, y_test5.shape)


(33783, 633) (14479, 633) (33783, 1) (14479, 1)


In [ ]:
results5 = {}
baseline(y_test5, results5)

Dumb baseline AUC 0.5
maximum f1 score, thres 0.49689009347093366 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5 0.49689009347093366 0.987637267767111 0.5 0.012362732232889011 0.0
[[14300     0]
 [  179     0]]


In [ ]:
stats_only(x_train5, y_train5, x_test5, y_test5, results5)



running xgb...
[10, 20, 65]
Train AUC:  0.9849488580069498
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6748674005110863 0.7
auc, f1, accu, accu_bl, precision, recall=  0.9743772707739188 0.6748674005110863 0.9796256647558533 0.7248855334609525 0.14310558831587566 0.46368715083798884
[[14101   199]
 [   96    83]]


In [65]:
nlp_only(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.8989277296190276
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.559744135036995 0.4
auc, f1, accu, accu_bl, precision, recall=  0.7471119662460444 0.5116270255233326 0.9675391946957663 0.591883423838731 0.030760854237860832 0.20670391061452514
[[13972   328]
 [  142    37]]


In [66]:
weather_only(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.999511544138674
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 20}
maximum f1 score, thres 0.5343266790851295 0.5
auc, f1, accu, accu_bl, precision, recall=  0.6996911747470407 0.5182796030688209 0.9864631535327025 0.521472242840958 0.022644810900658997 0.0446927374301676
[[14275    25]
 [  171     8]]


In [67]:
weather_nlp(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.9808870702180577
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 65}
maximum f1 score, thres 0.5683454486568243 0.7
auc, f1, accu, accu_bl, precision, recall=  0.722480173457827 0.5683454486568243 0.9685751778437738 0.6034412235808884 0.03576306019215456 0.22905027932960895
[[13983   317]
 [  138    41]]


In [68]:
stats_nlp(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.9930672341728458
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6420605551907898 0.5
auc, f1, accu, accu_bl, precision, recall=  0.9681359143649646 0.6369659980355152 0.9708543407693901 0.739753291401336 0.11312413038513905 0.5027932960893855
[[13967   333]
 [   89    90]]


In [69]:
stats_weather(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.9999705727729907
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.6201913982089219 0.4
auc, f1, accu, accu_bl, precision, recall=  0.9670244559909364 0.5717795619100151 0.9797637958422543 0.6311722076805876 0.07224931222563563 0.2737430167597765
[[14137   163]
 [  130    49]]


In [70]:
all_vars(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.9999831380183886
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.596686619458772 0.5
auc, f1, accu, accu_bl, precision, recall=  0.9638748681486111 0.558566881820114 0.9820429587678707 0.5881927569637067 0.051469956264290045 0.18435754189944134
[[14186   114]
 [  146    33]]
